# Discord messenger

In [1]:
import nest_asyncio
nest_asyncio.apply()
import discord
from discord.ext import tasks, commands
from datetime import datetime
import json

from binance.client import Client
from binance.enums import * #https://github.com/sammchardy/python-binance/blob/master/binance/enums.py

def get_api_keys(site: str, api_type: str)->str:
    """
    gets api keys stored in api-keys/api-keys.txt
    site: 'binance'
    api_type: 'api', 'secret'
    """
    with open('../api-keys/api-keys.txt') as json_file:
        return json.load(json_file)[site][api_type]
# Constants
dtoken = get_api_keys("discord", "api")

bot = commands.Bot(command_prefix="-")

api_key = get_api_keys("binance", "api")
api_secret = get_api_keys("binance", "secret")

client = Client(api_key=api_key, api_secret=api_secret)

In [2]:
messages = {"message":"", "o_l":"", "o_s":""}
def get_message(message:str):
    global messages
    """gets the message"""
    with open('message.txt') as json_file:
        new_message = json.load(json_file)[message]
        try: #Basically, sometimes if you don't need to limit rate
            if messages[message] == new_message: #Same message
                return ""
            elif messages[message] == "": #Haven't updated
                messages[message] = new_message
                return ""
            messages[message] = new_message
        except:
            pass
        return new_message

In [ ]:
@bot.event
async def on_ready():
    print("Logged in")
    updater.start()


@tasks.loop(seconds=60)
async def updater():
    noti = bot.get_channel(853110820611555328)
    announcements = bot.get_channel(864949029623169044)
#     await channel.send(str(datetime.today()))
    try:
        await announcements.send(get_message("message"))
    except:
        pass
    try:
        await noti.send(get_message("o_l"))
    except:
        pass
    try:
        await noti.send(get_message("o_s"))
    except:
        pass
    
@bot.command()
async def portfolio(message, asset="all"):
    '''Returns current USDT, ZEC, and XMR amounts'''
    if asset.lower() == "usdt":
        await message.channel.send(get_message('USDT'))
    elif asset.lower() == "xmr":
        await message.channel.send(get_message('XMR'))
    elif asset.lower() == "zec":
        await message.channel.send(get_message('ZEC'))
    elif asset.lower() == "all":
        await message.channel.send(get_message('USDT'))
        await message.channel.send(get_message('XMR'))
        await message.channel.send(get_message('ZEC'))
    else:
        await message.channel.send("There are no positions with that asset")
        
@bot.command()
async def mode(message):
    '''Returns current printer mode, which corresponds to printer refersh rate'''
    await message.channel.send(f"Current mode is : {get_message('mode')}")
    
@bot.command()
async def z(message):
    '''Returns current z-score'''
    await message.channel.send(f"Latest z-score: {str(round(float(get_message('z')), 2))}")

@bot.command()
async def level(message):
    """Get current margin level, where 2 or greater is safe"""
    await message.channel.send(f"Current margin level is {client.get_margin_account()['marginLevel']}")
    
@bot.command()
async def position(message, asset="ALL"):
    """Get the position of your trades and current unrealized gains/losses. ASSUMES there is an ongoing trade"""
    trade = 0 #Price that I traded the asset for
    price = 0 #Current price
    position = ""
    percent = 0
    if asset.upper() == "ZEC" or asset.upper() == "ZECUSDT" or asset.upper() == "ALL":
        price = round(get_price("ZECUSDT"), 2)
        trade = round(float(client.get_margin_trades(symbol='ZECUSDT')[-1]["price"]), 2)
        if client.get_margin_trades(symbol='ZECUSDT')[-1]["isBuyer"]:
            position = "Long"
            percent = round((price-trade)/trade, 2)
        else:
            position = "Short"
            percent = round((trade-price)/trade, 2)
        await message.channel.send(f"{position} ZEC: Trade price: {str(trade)}, now: {str(price)}, current gain/loss: {str(percent)}%")
    if asset.upper() == "XMR" or asset.upper() == "XMRUSDT" or asset.upper() == "ALL":
        price = round(get_price("XMRUSDT"), 2)
        trade = round(float(client.get_margin_trades(symbol='XMRUSDT')[-1]["price"]), 2)
        if client.get_margin_trades(symbol='XMRUSDT')[-1]["isBuyer"]:
            position = "Long"
            percent = round((price-trade)/trade, 2)
        else:
            position = "Short"
            percent = round((trade-price)/trade, 2)
        await message.channel.send(f"{position} XMR: Trade price: {str(trade)}, now: {str(price)}, current gain/loss: {str(percent)}%")

@updater.before_loop
async def before():
    await bot.wait_until_ready()
    
def get_price(symbol:str):
    """returns the price. symbol MUST include USDT, ie ZECUSDT"""
    return float(client.get_recent_trades(symbol=symbol, limit=1)[0]["price"])

bot.run(dtoken)

Logged in
